## Import necessary libraries & define our functions

In [ ]:
import json
import pickle
import pandas as pd
import cv2
import os
from PIL import Image, ImageDraw, ImageFilter, ImageEnhance
import imagehash
from urllib.request import urlopen
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import numpy as np
import random
import os
from tensorflow.keras import datasets, layers, models
import shutil
from time import localtime, strftime, time
from natsort import natsorted
from IPython.display import display

def current_time():
    '''Help: Returns the current time as a nice string.'''
    return strftime("%B %d, %-I:%M%p", localtime())

def elapsed_time(start_time):
    '''Using seconds since epoch, determine how much time has passed since the provided float. Returns string
    with hours:minutes:seconds'''
    elapsed_seconds = time()-start_time
    h = int(elapsed_seconds/3600)
    m = int((elapsed_seconds-h*3600)/60)
    s = int((elapsed_seconds-m*60)-h*3600)
    return f'{h}hr {m}m {s}s'

#simple function to pickle variables for later use. save a local pickle
def save_object(obj, filename, verbose=True):
    '''Help: Given an object & filepath, store the object as a pickle for later use.'''
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    if verbose:
        print(f"File saved at {filename}")
    
#and later load the file back into a variable
def load_object(filename, verbose=True):
    '''Help: Loads something previously pickled from the provided file path.'''
    with open(filename, 'rb') as f:
        load_test = pickle.load(f)
    if verbose:
        print(f"File loaded from {filename}")
    return load_test

def zoom_rotate_img(image):
    '''Help: Randomly rotate and zoom the given PIL image degrees and return it'''
    #store initial image size
    initial_size = image.size
    #determine at random how much or little we scale the image
    scale = 0.95+random.random()*.1
    scaled_img_size = tuple([int(i*scale) for i in initial_size])


    #create a blank background with a random color and same size as intial image
    bg_color = tuple(np.random.choice(range(256),size=3))
    background = Image.new('RGB', initial_size, bg_color)

    #determine the center location to place our rotated card
    center_box = tuple((n-o)//2 for n,o in zip(initial_size, scaled_img_size))

    #scale the image
    scaled_img = image.resize(scaled_img_size)

    #randomly select an angle to skew the image
    max_angle = 5
    skew_angle = random.randint(-max_angle, max_angle)
    
    #add the scaled image to our color background
    background.paste(scaled_img.rotate(skew_angle, fillcolor=bg_color,expand=1).resize(scaled_img_size), 
                     center_box)

    #potentially flip the image 180 degrees
    if random.choice([True, False]):
        background = background.rotate(180)
    
    return background

def blur_img(image):
    '''Help: Blur the given PIL image and return it'''
    return image.filter(filter=ImageFilter.BLUR)

def adjust_color(image):
    '''Help: Randomly reduce or increase the saturation of the provided image and return it'''
    converter = ImageEnhance.Color(image)
    #randomly decide to half or double the image saturation
    saturation = random.choice([.5, 1.5])
    return converter.enhance(saturation)

def adjust_contrast(image):
    '''Help: Randomly decrease or increase the contrast of the provided image and return it'''
    converter = ImageEnhance.Contrast(image)
    #randomly decide to half or double the image saturation
    contrast = random.choice([.5, 1.5])
    return converter.enhance(contrast)

def adjust_sharpness(image):
    '''Help: Randomly decrease or increase the sharpness of the provided image and return it'''
    converter = ImageEnhance.Sharpness(image)
    #randomly decide to half or double the image saturation
    sharpness = random.choice([.5, 1.5])
    return converter.enhance(sharpness)

def random_edit_img(image, distort=True, verbose=True):
    '''Help: Make poor edits to the image at random and return the finished copy. Can optionally not distort
    the image if need be.'''
    
    if distort:
        #randomly choose which editing operations to perform
        edit_permission = np.random.choice(a=[False, True], size=(4))

        #always skew the image, randomly make the other edits
        image = zoom_rotate_img(image)
        if verbose:
            print('Image skewed')
        if edit_permission[0]:
            image = blur_img(image)
            if verbose:
                print('Image blurred')    
        if edit_permission[1]:
            image = adjust_color(image)
            if verbose:
                print('Image color adjusted')
        if edit_permission[2]:
            image = adjust_contrast(image)
            if verbose:
                print('Image contrast adjusted')
        if edit_permission[3]:
            image = adjust_sharpness(image)
            if verbose:
                print('Image sharpness adjusted')
    
    return image


def generate_distorted_imgs(multiverse_id_list, num_distortions, num_undistorted, storage_path, \
                            resize=False, img_size=(224,312)):
    '''Help: High level function to reduce the testing & training image creation process down to a single step. 
    Provide a string list of multiverse_ids, the number of distortions desired for each printing, a general 
    storage location for the image files, and the final image size desired. Creates "poorly photographed" 
    versions of each multiverse_id printing provided. Results are named based on their index in the list.'''
    
    image_size = 'large'
    
    images_created = 0
    printings_distorted = 0
    #iterate through the provided list
    for multiverse_id in multiverse_id_list:
        printings_distorted += 1
        #pull the image URL using the multiverse_id
        image_url = modified_light_df_en[modified_light_df_en['multiverse_ids']\
                                         == int(multiverse_id)]['image_uris'].values[0][image_size]
        #get the raw image file
        clean_img = Image.open(urlopen(image_url))

        #use the index instead of multiverse_id in filename
        print_index = multiverse_id_list.index(multiverse_id)

        #now produce poorly rendered versions of the printing
        for i in range(num_distortions):
            images_created += 1
            if i < num_undistorted:
                distorted_img = random_edit_img(clean_img, distort=False, verbose=False)
            else:
                distorted_img = random_edit_img(clean_img, verbose=False)

            #potentially resize the image and save it locally #.resize((224,312))
            if resize:
                distorted_img = distorted_img.resize(img_size)
                
            distorted_img.save(f"{storage_path}/{print_index}-{i}.jpg")
            
    print(f"\n{images_created} total unique distortions saved from {printings_distorted} different printings.")
    print(f"Images stored @ '{storage_path}'\n")



def prep_images_for_network(storage_path):
    '''Help: Given a folder of distorted printings, compile all images and their labels into arrays for
    neural network processing. Returns image_array, label_array'''
    
    #initialize the empty arrays
    image_array = []
    label_array = np.array([], dtype=int)

    for subdir, dirs, files in os.walk(storage_path):
        for file in np.sort(files):
            if file.endswith('.jpg'):
                #open the image, then convert it to an array and scale values from 0-1
                image = Image.open(f"{storage_path}/{file}")
                scaled_array = np.array(image)/255

                #pull the multiverse_id from the filename
                label = int(file.split('-')[0])

                #add the data
                image_array.append(scaled_array)
                label_array = np.append(label_array, label)

    #convert image list to numpy array
    image_array = np.array(image_array)
    
    return image_array, label_array


#single code block to generate distorted images, prep them for training, and save the arrays. all work is saved 
#in a new directory that is created for this run. also saves a description of the model

def generate_img_set(image_set_name, multiverse_id_list, num_distortions, resize=True, verbose=True):
    '''Help: Given appropriate parameters, generate num_distortions distorted image copies of each card in 
    multiverse_id_list. Then prep all the images for neural net training and save the resulting arrays.
    Returns model_data: ((training_images, training_labels), (testing_images, testing_labels))
    
    image_set_name: str, desired folder name of current image set
    multiverse_ids_list, list of ints, card printings to use
    num_distortions, number of warped copies of each card to create
    resize, boolean, if false, images keep 936,672 original resolution, otherwise resize to (224,312)
    verbose, boolean, if true print statements show function progress
    '''

    if verbose:
        print(f"Process started for {image_set_name} on {current_time()} ...")
        start_time = time()

    #if the folder already exists, delete it so we can start fresh
    if os.path.exists(image_set_name):
        shutil.rmtree(image_set_name)

    #now create the directory, and sub folders for image storage
    os.mkdir(image_set_name)
    os.mkdir(f'{image_set_name}/Testing')
    os.mkdir(f'{image_set_name}/Training')

    if verbose:
        print(f'Folder structure created, generating {len(multiverse_id_list)*num_distortions} \
training images now ...')

    #now create images for training and testing, testing will always have two images, change here if need be
    #create the training images
    training_storage_path = f"{image_set_name}/Training"
    num_undistorted = 3
    generate_distorted_imgs(multiverse_id_list, num_distortions, num_undistorted, training_storage_path, resize)

    if verbose:
        print(f"Training images finished on {current_time()}, now generating {len(multiverse_id_list)*2} \
testing images ...")

    #create the testing images
    testing_storage_path = f"{image_set_name}/Testing"
    num_undistorted = 1
    generate_distorted_imgs(multiverse_id_list, 2, num_undistorted, testing_storage_path, resize)

    if verbose:
        print(f"All images created and saved under {image_set_name} on {current_time()}. \n\
Formatting images and labels for neural net processing now ...")

    #now open up all the image files and store contents as arrays for the neural net
    training_images, training_labels = prep_images_for_network(training_storage_path)
    testing_images, testing_labels = prep_images_for_network(testing_storage_path)

    #save the input arrays locally for later use in case we want them
    model_data = ((training_images, training_labels), (testing_images, testing_labels))
    save_object(model_data, f'{image_set_name} Arrays.p', verbose=False)

    if verbose:
        print(f"Pre processing complete on {current_time()} after {elapsed_time(start_time)}. \
\n\nTraining & testing data saved locally ({image_set_name} Arrays.p) and ready for neural network!\n\n")

    return model_data


def train_CNN_model(model_name, model_data, unique_printings, epochs=10, verbose=True):
    '''Help: Create and train a CNN model for the provided model_data'''
    
    #unpack the model_data variable
    ((training_images, training_labels), (testing_images, testing_labels)) = model_data

    if verbose:
        print(f'Initializing {model_name} on {current_time()} ...')
        model_start_time = time()

    #if the folder already exists, delete it so we can start fresh
    if os.path.exists(f'{model_name}.model'):
        shutil.rmtree(f'{model_name}.model')

    #initialize the neural network model
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=training_images.shape[1:]))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(unique_printings, activation='softmax'))

    #compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    if verbose:
        print('Network compiled, fitting data now ... \n')
    #fit the model to the provided data
    model.fit(training_images, training_labels, epochs=epochs, validation_data=(testing_images, testing_labels))

    if verbose:
        print('\nModel fit, elvaluating accuracy and saving locally now ... \n')
    #evaluate the model

    loss, accuracy = model.evaluate(testing_images, testing_labels)
    print(f'Loss: {loss}')
    print(f'Accuracy: {accuracy}')

    #save it locally for future reuse
    model.save(f'{model_name}.model')

    if verbose:
        print(f"\nModel evaluated & saved locally at '{model_name}.model' on {current_time()} \
after {elapsed_time(model_start_time)}!\n")

    return model


def test_model_via_index(image_set_name, card_index, model, sub_index = 0):
    filepath = f'{image_set_name}/Testing/{card_index}-{sub_index}.jpg'
    test_card = Image.open(filepath)

    #provide the image to the model and see what comes back
    img_as_array = np.array(np.array(test_card)/255)

    eval_images = []
    eval_images.append(img_as_array)
    eval_images = np.array(eval_images)

    result = model.predict(eval_images)
    result_index, confidence = np.argmax(result), result[0,np.argmax(result)]

    correct = False
    #display the result!
    if result_index == card_index:
        #display(test_card)
        correct = True
        print(f'For card index {card_index}, model predicted index {result_index} \
with {np.round(confidence*100,4)}% confidence.')
    
    else:
        print(f'For card index {card_index}, model predicted index {result_index} \
with {np.round(confidence*100,4)}% confidence. (INCORRECT)')
        #display(test_card, Image.open(f'{image_set_name}/Testing/{result_index}-sub_index.jpg'))

    

## Load the modified scryfall database of cards, and the list subset of valid options

In [ ]:
#load english card database where multiverse_ids have been changed from lists to a single int value
modified_light_df_en = load_object('modified_light_df_en.p')

#and load the accompanying list of multiverse_ids suitable to use
valid_multiverse_ids = load_object('Valid Multiverse IDs.p')


## Now select at random a few cards to use as a sample set

In [ ]:
#choose at random 100 of those cards sorted numerically by multiverse_id
num_unique_prints = 100
chosen_multiverse_ids = natsorted(random.sample(valid_multiverse_ids, num_unique_prints))


## Create training & testing data from the chosen card set

In [ ]:
#define necessary variables and prepare all images for neural network training

image_set_name = 'Demo Sample'
multiverse_id_list = chosen_multiverse_ids
num_distortions = 10
resize = True
verbose = True

demo_model_data = generate_img_set(image_set_name, multiverse_id_list, num_distortions, resize, verbose)


## Train a neural network to recognize the cards

In [ ]:
#provide information about the model and train it!
model_name = 'Demo Sample Model'
epochs = 10
verbose = True

demo_model = train_CNN_model(model_name, demo_model_data, num_unique_prints, epochs, verbose)


## Double check the results 

In [ ]:
#go through all testing images & print results
for i in range(num_unique_prints):
    test_model_via_index(image_set_name, i, demo_model, sub_index = 0)
    

<br>

# Let's try using the same sample set as before

<br><br>


In [ ]:
#load the multiverse_ids of the cards used in the previous demo
prior_multiverse_ids = load_object('prior_multiverse_ids_used.p')


In [ ]:
#define necessary variables and prepare all images for neural network training

image_set_name = 'Prior Card Sample'
multiverse_id_list = prior_multiverse_ids
num_distortions = 10
resize = True
verbose = True

prior_card_model_data = generate_img_set(image_set_name, multiverse_id_list, num_distortions, resize, verbose)


In [ ]:
#provide information about the model and train it!
model_name = 'Prior Card Sample Model'
epochs = 10
verbose = True
num_unique_prints = len(multiverse_id_list)

prior_card_model = train_CNN_model(model_name, prior_card_model_data, num_unique_prints, epochs, verbose)


In [ ]:
#go through all testing images & print results
for i in range(num_unique_prints):
    test_model_via_index(image_set_name, i, prior_card_model, sub_index = 0)
    

## Thanks for watching, stay tuned for next updates!
<br>